In [47]:
import constantes as cons
import sys
import pandas as pd
from backtesting import Backtest, Strategy
import pandas_ta as ta
import talib
import numpy as np

symbol =  str.upper('BEL'+'USDT')
mult_take_profit = 1
mult_stop_loss = 1

def get_bollinger_bands(df):
    mult = 2.0
    length = 20
    # calcular indicadores
    close = df['Close']
    basis = talib.SMA(close, length)
    dev = mult * talib.STDDEV(close, length)
    df['upper'] = basis + dev
    df['lower'] = basis - dev
    # imprimir resultados
    return df

def obtiene_historial(symbol):
    client = cons.client
    timeframe='30m'
    leido=False
    while leido==False:
        try:
            historical_data = client.get_historical_klines(symbol, timeframe)
            leido = True
        except KeyboardInterrupt as ky:
            print("\nSalida solicitada. ")
            sys.exit()              
        except:
            print("Intento leer de nuevo...")
            pass
    data = pd.DataFrame(historical_data)
    data.columns = ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote Asset Volume', 
                        'Number of Trades', 'TB Base Volume', 'TB Quote Volume', 'Ignore']
    data['Open Time'] = pd.to_datetime(data['Open Time']/1000, unit='s')
    data['Close Time'] = pd.to_datetime(data['Close Time']/1000, unit='s')
    numeric_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote Asset Volume', 'TB Base Volume', 'TB Quote Volume']
    data[numeric_columns] = data[numeric_columns].apply(pd.to_numeric, axis=1)
    data['timestamp']=data['Open Time']
    data.set_index('timestamp', inplace=True)
    data.dropna(inplace=True)
    data.drop(['Open Time','Close Time','Quote Asset Volume', 'TB Base Volume', 'TB Quote Volume','Number of Trades',
            'Ignore'], axis=1, inplace=True)    
    data['ema20']=data.ta.ema(20)
    data['ema50']=data.ta.ema(50)
    data['ema200']=data.ta.ema(200)
    data['atr']=ta.atr(data.High, data.Low, data.Close)
    data = get_bollinger_bands(data)
    return data

def EMA(data,length):
    return data.ta.ema(length)

def backtesting(data): 
    class Fenix(Strategy):
        def init(self):
            self.ema20 = self.I(EMA, data, 20, color="green")
            self.ema50 = self.I(EMA, data, 50,color="yellow")
            self.ema200 = self.I(EMA, data, 200, color="grey")
        def next(self):
            if not self.position:
                if self.data.signal[-1] ==1:
                    self.buy(size=1000, sl=self.data.stop_loss[-1] , tp=self.data.take_profit[-1])
                elif self.data.signal[-1] ==-1:
                    self.sell(size=1000, sl=self.data.stop_loss[-1] , tp=self.data.take_profit[-1])
            else:
                pass
    bt = Backtest(data, Fenix, cash=1000, commission=.002, exclusive_orders=True)
    output = bt.run()
    bt.plot()
    print(output)

def estrategia(data):
    data['signal']= np.where((data.ema20 > data.ema50) & (data.ema50 > data.ema200) & (data.Close.shift(1)<data.lower.shift(1)),1,
                    (np.where((data.ema20 < data.ema50) & (data.ema50 < data.ema200) & (data.Close.shift(1)>data.upper.shift(1)),-1,
                            0)))        
    data['take_profit'] =np.where(data.signal==1,data.Close+(data.atr*mult_take_profit) ,np.where(data.signal==-1,data.Close-(data.atr*mult_take_profit),0))
    data['stop_loss']   =np.where(data.signal==1,data.Close-(data.atr*mult_stop_loss)   ,np.where(data.signal==-1,data.Close+(data.atr*mult_stop_loss),0))
    return data

data = obtiene_historial(symbol)
data = estrategia(data)
backtesting(data)

C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2023-06-04 01:30:00
End                       2023-06-24 21:00:00
Duration                     20 days 19:30:00
Exposure Time [%]                         6.4
Equity Final [$]                  1000.334524
Equity Peak [$]                   1000.426035
Return [%]                           0.033452
Buy & Hold Return [%]              -16.867583
Return (Ann.) [%]                     0.58303
Volatility (Ann.) [%]                1.416164
Sharpe Ratio                         0.411697
Sortino Ratio                        0.596755
Calmar Ratio                         2.104454
Max. Drawdown [%]                   -0.277046
Avg. Drawdown [%]                   -0.158392
Max. Drawdown Duration        9 days 12:00:00
Avg. Drawdown Duration        4 days 20:40:00
# Trades                                   20
Win Rate [%]                             60.0
Best Trade [%]                       1.848017
Worst Trade [%]                     -3.170477
Avg. Trade [%]                    